<h3> Wildfire Web Map </h3> 
<p> This script will allow for a webmap to be created for any wildfires that occur during the 2021 season.</p> 
<p> The Source for this webmap comes from this Github Repo <a href ="https://github.com/achapkowski/gathering_license_info/blob/main/Headless%20WebMap%20Management.ipynb"> Headless Web Map </a> <p> Goals of this Script: 
    <ul> 1.) Develop Web Map that shows the boundry of a wildfire. </ul> 
    <ul> 2.) Add Layers: Counties, Road Closures </ul>
    <ul> 3.) Measure Size of Fire </ul> 
    <ul> 4.) ** Determind Critical Infrastructure Within a Five Mile Raidus of the Fire </ul> 
    <ul> ** If possible, this maybe be a future addition to this script. <ul/> <p/> 

In [2]:
# Step 1 import packages 
import pandas as pd 
import numpy as pd 
import arcgis # import gis is the command for import the ArcGIS API for python. 
import os 
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.mapping import MapServiceLayer

<p><b> Create a GIS that can publish to your org</b></p> 

In [3]:
# Note:you will need your arcgis online account for this. 
# in the notebook example, it says to use your profile, I am not sure what this is. 
map =  GIS(url= 'https://dhsem.maps.arcgis.com/home/index.html', username='rstevenson88@DHSEM')
# this will trigger your password to be entered, without having to share it. 
print("arcgis account username:")

Enter password: ········
arcgis account username:


<p><b> Create an Empty Webmap<b><p/>

In [4]:
wm = WebMap()

<p><b> Select a basemap to add </b></p> 

In [5]:
# Lets see what the default basemaps that are avialable and start from there. 
print(wm.basemaps)

['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oceans', 'osm', 'satellite', 'streets', 'streets-navigation-vector', 'streets-night-vector', 'streets-relief-vector', 'streets-vector', 'terrain', 'topo', 'topo-vector']


In [6]:
#Select a basemap Please note that if you want labels, you'll need to use one of the basemap choices with the word vector at the end. 
wm.basemap = 'dark-gray-vector'

<p><b> Add your Layers for wildfires</b></p>
<p> Layers needed: 
    <ul> 1.) Counties of Colorado </ul> 
    <ul> 2.) Wildfire Perimeter: you can find the fire perimeters at <a href ='https://data-nifc.opendata.arcgis.com/datasets/2021-wildland-fire-perimeters-to-date?geometry=101.953%2C-88.438%2C-101.953%2C88.438'> the following site.</a> </ul>
    <ul> 3.) Other layers as needed (evaucation sites, shelter locations, etc) </ul> </p> 

In [37]:
# Add Layers Note: Every layer that is loaded into AGOL has a API url to access that layer. 
counties = MapServiceLayer(url='https://services1.arcgis.com/4Y4MvWjDdqrxtGBH/ArcGIS/rest/services/Colorado_Counties/FeatureServer/0')
FP =  MapServiceLayer(url="https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/CY_WildlandFire_Perimeters_ToDate/FeatureServer/0")
wm.add_layer(counties)
wm.add_layer(FP)
#  Note: you added the 2021 Wildland Fire Perimeters to Date. 
#  First: this is the fire perimeter for 2021 data, if you use this notebook in the future lets say 2022, you may need to update the URL. 
#  Second: You have added the feature layer, but you will need to run a query to specficy which wildfire your are interested in working with. 

True

<p><b> Query The FP Layer to select a specfic wildfire </b></p> 

In [8]:
# Get attributes of the FP layer 
for f in FP.properties.fields:
    print(f['name'])
#  So here are all the attrbutes for the FP mapservice layer. 
# Now lets try to query irwin_IncidentName and see what we get. 

OBJECTID
poly_IncidentName
poly_FeatureCategory
poly_MapMethod
poly_GISAcres
poly_CreateDate
poly_DateCurrent
poly_PolygonDateTime
poly_Acres_AutoCalc
poly_GlobalID
poly_Source
irwin_ABCDMisc
irwin_ADSPermissionState
irwin_CalculatedAcres
irwin_ContainmentDateTime
irwin_ControlDateTime
irwin_DailyAcres
irwin_DiscoveryAcres
irwin_DispatchCenterID
irwin_FFReportApprovedByTitle
irwin_FFReportApprovedByUnit
irwin_FFReportApprovedDate
irwin_FireBehaviorGeneral
irwin_FireBehaviorGeneral1
irwin_FireBehaviorGeneral2
irwin_FireBehaviorGeneral3
irwin_FireCause
irwin_FireCauseGeneral
irwin_FireCauseSpecific
irwin_FireCode
irwin_FireDepartmentID
irwin_FireDiscoveryDateTime
irwin_FireMgmtComplexity
irwin_FireOutDateTime
irwin_FSConfinePercent
irwin_FSFullSuppPercent
irwin_FSMonitorPercent
irwin_FSPointZonePercent
irwin_FSJobCode
irwin_FSOverrideCode
irwin_GACC
irwin_ICS209ReportDateTime
irwin_ICS209RForTimePeriodFrom
irwin_ICS209RForTimePeriodTo
irwin_ICS209ReportStatus
irwin_IncidentManagementOrg


In [27]:
# This query doesnt work, yet. I need to figure out why the paramters isnt correct. 
#queryFP = FP.query(where="irwin_IncidentName==CROWS NEST") # I dont know why I cant query string  attributes?? is it two equals or one? 
queryFP = FP.query(where="irwin_POOFips=08057") # There are two wildfires in this county. 
print(queryFP)

{"features": [{"geometry": {"rings": [[[-106.301651647647, 40.7223373486633], [-106.30163166642, 40.7223533335223], [-106.30159333351, 40.722338333803], [-106.301541667805, 40.7223316672599], [-106.301516711708, 40.7223008406172], [-106.301525826015, 40.7223003415794], [-106.301651647647, 40.7223373486633]], [[-106.301651647647, 40.7223373486633], [-106.301665000205, 40.7223266659906], [-106.301678730056, 40.7223241238261], [-106.301713960185, 40.7223302164426], [-106.30171999866, 40.722348334297], [-106.301715683154, 40.7223561834141], [-106.301651647647, 40.7223373486633]], [[-106.299931453828, 40.7240174916286], [-106.299930000354, 40.7240183337758], [-106.299936799702, 40.7239960001924], [-106.299931453828, 40.7240174916286]], [[-106.301516711708, 40.7223008406172], [-106.301501666724, 40.7223016657657], [-106.301466666564, 40.7223083336734], [-106.301416667233, 40.7223183328103], [-106.301378334323, 40.7223216667634], [-106.301336666867, 40.7223499995708], [-106.301323334072, 40.7

In [32]:
# Add Query As its own layer 
wm.add_layer(queryFP)

True

In [33]:
#Save Item to org DHSEM in our case 
item = wm.save(item_properties={
    'title' : "Wildfires in 2021",
    'snippet' : "dispalys Wildfires in Colorado",
    'tags' : 'wildfires'}
)

In [34]:
# View your webmap 
item.homepage

'https://DHSEM.maps.arcgis.com/home/item.html?id=e8761bddba42400298fd294f474cfa07'

<h3> Complete and  Future Changes </h3> 
<p> So Yay this script works, thats exciting! There are some tweaks that I want to make in the future:
    <ul> 1.) Figure out how to Query the Wildfire Feature Service when I bring it into the Notebook. </ul> 
    <ul> 2.) Create an Extent for the Webmap to Zoom into the wildfire in Question. </ul>
    <ul> 3.) Figure out Why when I look at this map in the classic map viewer no layers show up in the contents pane.. but layers show up on the map its self? odd. </ul> 
    <ul> 4.) Get feedback about what additional layers should be added to this map for wildfire season. </ul> 